# Testing functionality of Last Writer Wins Element Set with CRDT Implementation

This code and document is made by Alexander Kahanek on 3/31/2021.

Resources used:

+ https://www.youtube.com/watch?v=iEFcmfmdh2w 
+ https://www.youtube.com/watch?v=OOlnp2bZVRs 
+ https://hal.inria.fr/inria-00555588/PDF/techreport.pdf 

## LWW Element Set Implementation

The functionality of the LWW set is that you can assign LWW graphs to each client, and be able to change elements (add or remove) without worry of them getting mixed up due to time or update issues. This allows for offline and asynchronous client side modification of elements, yet when the LWW graphs link up everything merges correctly without conflict.

In this scenario, each LWW set has as many elements as it wants, which can be added or removed, and keeps track of two sets of elements. Both an Added (A) set and a Removed (R) set. Then the final kept set is determined by timestamps (Hybrid Logical Clock (HLC) seems to be best, but a simple implementation was not working for me locally). Effectively, if an element exists in A only, then it is added; however, if it exists in A and R, then you base it on the timestamp. If the timestamp is more recent for the element in R, then it is not included, else it is included. The benefits of this system is it sets up a clear winner for which elements are kept. Thus, when merging LWW graphs all you need to do is update all your elements to the max timestamp.

The outline of the entire use case is:

+ set up LWW graphs
    - these graphs can be enabled for two clients

+ add / remove elements
    - keep all unique elements in both add and remove with most recent timestamp

+ merge graphs on client sync
    - update all unique elements to the most recent time

This allows you to get the most recent client side updates possible, then when the clients sync and graphs merge, it allows you to properly translate the offline add / removes to the true most current graph.

## Testing

Here we are doing simple tests to make sure the add and remove implementation works correctly. For our LWW set we will do the following:

+ L1 = a -> -a -> a
    - here (-a) signifies we are removing a
    - the final L1 set will be \[a]

In [4]:
from lww_set import *

# setup graphs
L1 = LWW(1)

# test functionality of adding and removing
print('here we are adding element "a", removing it, then adding it.')
L1.add("a")
print(L1.get_current()) # should be [a]
L1.remove("a")
print(L1.get_current()) # should be []
L1.add("a")
print(L1.get_current()) # should be [a]

here we are adding element "a", removing it, then adding it.
[{'element': 'a', 'timestamp': 2793.2002573}]
[]
[{'element': 'a', 'timestamp': 2793.2004221}]


The above worked correctly. We can see the element being added, removed, then again added.

Lets try comparing, merging, and removing a node from a different graph. We will add nodes as:

+ L1 = a -> b -> c
+ L2 = d -> e -> -a
+ L1 will be \[a, b, c] and L2 will be \[d, e, -a]
+ when merged they will both be \[b, c, d, e]

In [13]:
# setup graphs
L1 = LWW(1)
L2 = LWW(2)

print('here we are adding elements [a, b, c] to Graph 1, and [d, e, -a] to Graph 2')
L1.add("a")
L1.add("b")
L1.add("c")
L2.add("d")
L2.add("e")
L2.remove("a")
print('comparison of LWW Graph 1 and 2:', L1.compare(L2)) # should be false
print(L1.get_current()) # should be [a, b, c]
print(L2.get_current()) # should be [d, e, a]

print()
print('now we are merging Graph 1 and 2 together ... ')
L1.merge(L2)
print('comparison of LWW Graph 1 and 2:', L1.compare(L2)) # should be true
print(L1.get_current()) # should be [a, b, c, d, e]
print(L2.get_current()) # should be [a, b, c, d, e]

here we are adding elements [a, b, c] to Graph 1, and [d, e, -a] to Graph 2
comparison of LWW Graph 1 and 2: False
[{'element': 'a', 'timestamp': 3526.808861}, {'element': 'b', 'timestamp': 3526.8088933}, {'element': 'c', 'timestamp': 3526.8089212}]
[{'element': 'd', 'timestamp': 3526.8089473}, {'element': 'e', 'timestamp': 3526.8089725}]

now we are merging Graph 1 and 2 together ... 
comparison of LWW Graph 1 and 2: True
[{'element': 'b', 'timestamp': 3526.8088933}, {'element': 'c', 'timestamp': 3526.8089212}, {'element': 'd', 'timestamp': 3526.8089473}, {'element': 'e', 'timestamp': 3526.8089725}]
[{'element': 'b', 'timestamp': 3526.8088933}, {'element': 'c', 'timestamp': 3526.8089212}, {'element': 'd', 'timestamp': 3526.8089473}, {'element': 'e', 'timestamp': 3526.8089725}]


Finally, lets simulate a real task. I will define a list of elements, then have three clients randomly choose which to add or remove, update 2 of them, then add another client and randomly choose which ones to add or remove.

In [33]:
import random
elements = ["a", "b", "c", "d", "e", "f", {"dog":"link"}, 5, "alexander kahanek"]

L1 = LWW(1)
L2 = LWW(2)
L3 = LWW(3)

graphs = [L1, L2, L3]

for i in range(5):
    g = random.randint(0, len(graphs)-1) # random graph choice
    e = random.randint(0, len(elements)-1) # random element choice

    rand_graph = graphs[g]
    

    if random.randint(0,1) == 1:
        a_or_r = "+"
        rand_graph.add(elements[e])
    else:
        a_or_r = "-"
        rand_graph.remove(elements[e])

    print(f'L{g+1} -> {a_or_r} | {elements[e]}')

print()
print('-------------------------------------------')
print('updates after random choices')
print(f'L1 -> {L1.get_current()}')
print(f'L2 -> {L2.get_current()}')
print(f'L3 -> {L3.get_current()}')

print()
print('-------------------------------------------')
print('merging Graph 1 and 2 ...')
L1.merge(L2)
print(f'L1 -> {L1.get_current()}')
print(f'L2 -> {L2.get_current()}')
print(f'L3 -> {L3.get_current()}')

print()
print('adding Graph L4 and running more random choices')

L4 = LWW(4)
graphs.append(L4)

for i in range(10):
    g = random.randint(0, len(graphs)-1) # random graph choice
    e = random.randint(0, len(elements)-1) # random element choice

    rand_graph = graphs[g]
    

    if random.randint(0,1) == 1:
        a_or_r = "+"
        rand_graph.add(elements[e])
    else:
        a_or_r = "-"
        rand_graph.remove(elements[e])

    print(f'L{g+1} -> {a_or_r} | {elements[e]}')

print()
print('-------------------------------------------')
print('updates after random choices')
print(f'L1 -> {L1.get_current()}')
print(f'L2 -> {L2.get_current()}')
print(f'L3 -> {L3.get_current()}')
print(f'L4 -> {L4.get_current()}')

print()
print('-------------------------------------------')
print('merging Graph 1 <-> 3 and Graphs 2 <-> 4')
L1.merge(L3)
L2.merge(L4)
print(f'L1 -> {L1.get_current()}')
print(f'L2 -> {L2.get_current()}')
print(f'L3 -> {L3.get_current()}')
print(f'L4 -> {L4.get_current()}')

print()
print('-------------------------------------------')
print('merging all graphs')
L1.merge(L2)
L2.merge(L3)
L3.merge(L4)
print(f'L1 -> {L1.get_current()}')
print(f'L2 -> {L2.get_current()}')
print(f'L3 -> {L3.get_current()}')
print(f'L4 -> {L4.get_current()}')


L1 -> - | d
L1 -> + | a
L2 -> - | b
L3 -> + | d
L3 -> - | 5

-------------------------------------------
updates after random choices
L1 -> [{'element': 'a', 'timestamp': 5036.0707433}]
L2 -> []
L3 -> [{'element': 'd', 'timestamp': 5036.0707838}]

-------------------------------------------
merging Graph 1 and 2 ...
L1 -> [{'element': 'a', 'timestamp': 5036.0707433}]
L2 -> [{'element': 'a', 'timestamp': 5036.0707433}]
L3 -> [{'element': 'd', 'timestamp': 5036.0707838}]

adding Graph L4 and running more random choices
L2 -> - | d
L3 -> + | d
L2 -> + | c
L4 -> - | alexander kahanek
L3 -> - | c
L1 -> + | a
L3 -> + | c
L2 -> + | d
L4 -> + | a
L1 -> + | d

-------------------------------------------
updates after random choices
L1 -> [{'element': 'c', 'timestamp': 5036.072105}, {'element': 'a', 'timestamp': 5036.0721653}, {'element': 'd', 'timestamp': 5036.0722374}]
L2 -> [{'element': 'c', 'timestamp': 5036.072105}, {'element': 'a', 'timestamp': 5036.0721653}, {'element': 'd', 'timestamp': 

Here we see things working properly. It is hard to tell exactly because the two merges of Graphs 1 <-> 3 amd 2 <-> 4 are identical, but this is most likely due to just a low volume of choice and graphs. This would be much better with a lot more of each, but also exhausting to look at and figure out.

Remember, for this only the most recent changes to the elements matter. It does not matter what graph the updates happened on, which is the whole point of the LWW Element Set with CRDT implementation!

Thanks for reading, have a great day!